In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os

propublica_token = os.environ['propublicatoken']

memberurl = 'https://voteview.com/static/data/out/members/HS117_members.csv'
cvoteurl = 'https://voteview.com/static/data/out/rollcalls/HS117_rollcalls.csv'
memvoteurl = 'https://voteview.com/static/data/out/votes/HS117_votes.csv'
cpartyurl = 'https://voteview.com/static/data/out/parties/HS117_parties.csv'

members_vv = pd.read_csv(memberurl)
cvote_vv = pd.read_csv(cvoteurl)
memvote_vv = pd.read_csv(memvoteurl)
cparty_vv = pd.read_csv(cpartyurl)

useragent_url = 'https://httpbin.org/user-agent'
r = requests.get(useragent_url)
useragent = json.loads(r.text)['user-agent']

headers = {'X-API-Key': propublica_token,
          'User-Agent': useragent,
          'From': 'tby8aj@virginia.edu'}

root = 'https://api.propublica.org'
congress = '117'
chamber = 'house'
memberendpoint = '/congress/v1/{congress}/{chamber}/members.json'.format(congress=congress, chamber=chamber)

r = requests.get(root + memberendpoint, headers = headers)
myjson = json.loads(r.text)
house_pp = pd.json_normalize(myjson, record_path = ['results', 'members'])

chamber = 'senate'
memberendpoint = '/congress/v1/{congress}/{chamber}/members.json'.format(congress=congress, chamber=chamber)
r = requests.get(root + memberendpoint, headers = headers)
myjson = json.loads(r.text)
senate_pp = pd.json_normalize(myjson, record_path = ['results', 'members'])


billsendpoint = '/congress/v1/{congress}/{chamber}/bills/{billtype}.json'.format(congress='117',
                                                                                 chamber='both',
                                                                                 billtype='introduced')
n = 20
i = 0
bills_117 = pd.DataFrame()
while n == 20:
    myparams = {'query': '', 'offset':i}
    r = requests.get(root + billsendpoint, headers = headers, params = myparams)
    myjson = json.loads(r.text)
    bills_pp = pd.json_normalize(myjson, record_path = ['results', 'bills'])
    bills_117 = pd.concat([bills_117, bills_pp], ignore_index=True)
    i = i + 20
    print(i)
    n = myjson['results'][0]['num_results']

In [2]:
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',600)

members_vv.sort_values('nominate_dim1', ascending=True)

In [3]:
members_vv.query("chamber == 'Senate' & born >= 1980")

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,bioguide_id,born,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
473,117,Senate,42103,44,0,GA,100,NaN,NaN,"OSSOFF, Thomas Jonathan (Jon)",O000174,1987,NaN,-0.466,0.53,-23.94533,0.96961,776.0,10.0,NaN,-0.466,0.53


bills_117.groupby('sponsor_name').size().sort_values(ascending=False)

bills_117['sponsor_name'].value_counts()

type(bills_117['house_passage'][0])

bills_117['house_passage'][0] is not None

bills_117['pass_house'] = [x is not None for x in bills_117 ['house_passage']]
bills_117['pass_senate'] = [x is not None for x in bills_117 ['senate_passage']]
bills_117['pass_enact'] = [x is not None for x in bills_117 ['enacted']]

bills_117.groupby('sponsor_party').agg({'pass_house':'mean','pass_senate':'mean','pass_enact':'mean'})

In [4]:
members_vv.head(3)

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,bioguide_id,born,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,117,President,99912,99,0,USA,200,0.0,0.0,"TRUMP, Donald John",NaN,1946,NaN,0.403,0.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117,President,99913,99,0,USA,100,0.0,0.0,"BIDEN, Joseph Robinette, Jr.",B000444,1942,NaN,-0.320,0.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117,House,20301,41,3,AL,200,NaN,NaN,"ROGERS, Mike Dennis",R000575,1958,NaN,0.360,0.463,-74.23598,0.89924,699.0,24.0,NaN,0.422,0.539


In [5]:
house_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,A000370,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Alma,None,Adams,None,1946-05-27,F,D,None,RepAdams,CongresswomanAdams,None,412607,76386,5935,21545,N00035451,/m/02b45d,H4NC12100,https://adams.house.gov,https://adams.house.gov/rss.xml,None,True,None,-0.465,None,10,2022,925.0,3.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:nc/cd:12,2436 Rayburn House Office Building,202-225-1510,None,NC,12,False,3712,0.32,99.02,0.87
1,A000055,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,B.,Aderholt,None,1965-07-22,M,R,None,Robert_Aderholt,RobertAderholt,RobertAderholt,400004,45516,441,29701,N00003028,/m/024p03,H6AL04098,https://aderholt.house.gov,https://aderholt.house.gov/rss.xml,None,True,None,0.380,None,26,2022,925.0,14.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:al/cd:4,266 Cannon House Office Building,202-225-4876,None,AL,4,False,0104,1.51,96.24,3.65
2,A000371,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Pete,None,Aguilar,None,1979-06-19,M,D,House Democratic Caucus Vice Chair,reppeteaguilar,reppeteaguilar,None,412615,79994,70114,21506,N00033997,/m/0jwv0xf,H2CA31125,https://aguilar.house.gov,None,None,True,None,-0.296,None,8,2022,925.0,3.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:ca/cd:31,109 Cannon House Office Building,202-225-3201,None,CA,31,False,0631,0.32,98.80,1.09


In [6]:
senate_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,senate_class,state_rank,lis_id,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,B001230,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Tammy,None,Baldwin,None,1962-02-11,F,D,Senate Democratic Caucus Secretary,SenatorBaldwin,senatortammybaldwin,witammybaldwin,400013,57884,3470,29940,N00004367,/m/024v02,H8WI00018,https://www.baldwin.senate.gov,None,https://www.baldwin.senate.gov/feedback,True,None,-0.493,None,9,2024,866,1,0,2022-10-01 01:45:44 -0400,ocd-division/country:us/state:wi,709 Hart Senate Office Building,202-224-5653,None,WI,1,junior,S354,0.12,98.96,1.04
1,B001261,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,John,None,Barrasso,None,1952-07-21,M,R,Senate Republican Conference Chair,SenJohnBarrasso,johnbarrasso,barrassowyo,412251,1024777,52662,40707,N00006236,/m/02rsm32,S6WY00068,https://www.barrasso.senate.gov,https://www.barrasso.senate.gov/public/?a=rss....,https://www.barrasso.senate.gov/public/index.c...,True,None,0.537,None,15,2024,866,58,0,2022-10-01 01:45:43 -0400,ocd-division/country:us/state:wy,307 Dirksen Senate Office Building,202-224-6441,None,WY,1,senior,S317,6.70,95.33,4.67
2,B001267,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Michael,None,Bennet,None,1964-11-28,M,D,None,SenBennetCo,senbennetco,SenatorBennet,412330,1031622,110942,40910,N00030608,/m/05b60qf,S0CO00211,https://www.bennet.senate.gov/public,None,https://www.bennet.senate.gov/public/index.cfm...,True,None,-0.229,None,13,2022,866,19,0,2022-10-12 09:45:23 -0400,ocd-division/country:us/state:co,261 Russell Senate Office Building,202-224-5852,202-228-5097,CO,3,senior,S330,2.19,99.06,0.94


In [7]:
members_pp = pd.concat([senate_pp,house_pp],ignore_index=True)
#members_pp

In [8]:
members_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,senate_class,state_rank,lis_id,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,district,at_large,geoid
0,B001230,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Tammy,None,Baldwin,None,1962-02-11,F,D,Senate Democratic Caucus Secretary,SenatorBaldwin,senatortammybaldwin,witammybaldwin,400013,57884,3470,29940,N00004367,/m/024v02,H8WI00018,https://www.baldwin.senate.gov,None,https://www.baldwin.senate.gov/feedback,True,None,-0.493,None,9,2024,866.0,1.0,0.0,2022-10-01 01:45:44 -0400,ocd-division/country:us/state:wi,709 Hart Senate Office Building,202-224-5653,None,WI,1,junior,S354,0.12,98.96,1.04,NaN,NaN,NaN
1,B001261,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,John,None,Barrasso,None,1952-07-21,M,R,Senate Republican Conference Chair,SenJohnBarrasso,johnbarrasso,barrassowyo,412251,1024777,52662,40707,N00006236,/m/02rsm32,S6WY00068,https://www.barrasso.senate.gov,https://www.barrasso.senate.gov/public/?a=rss....,https://www.barrasso.senate.gov/public/index.c...,True,None,0.537,None,15,2024,866.0,58.0,0.0,2022-10-01 01:45:43 -0400,ocd-division/country:us/state:wy,307 Dirksen Senate Office Building,202-224-6441,None,WY,1,senior,S317,6.70,95.33,4.67,NaN,NaN,NaN
2,B001267,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Michael,None,Bennet,None,1964-11-28,M,D,None,SenBennetCo,senbennetco,SenatorBennet,412330,1031622,110942,40910,N00030608,/m/05b60qf,S0CO00211,https://www.bennet.senate.gov/public,None,https://www.bennet.senate.gov/public/index.cfm...,True,None,-0.229,None,13,2022,866.0,19.0,0.0,2022-10-12 09:45:23 -0400,ocd-division/country:us/state:co,261 Russell Senate Office Building,202-224-5852,202-228-5097,CO,3,senior,S330,2.19,99.06,0.94,NaN,NaN,NaN


In [9]:
members_pp.query("icpsr_id == '20301'")

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,senate_class,state_rank,lis_id,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,district,at_large,geoid
447,R000575,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Mike,D.,Rogers,None,1958-07-16,M,R,None,RepMikeRogersAL,CongressmanMikeDRogers,MikeRogersAL03,400341,1014740,5705,20301,N00024759,/m/024n_s,H2AL03032,https://mikerogers.house.gov,None,None,True,None,0.36,None,22,2022,925.0,37.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:al/cd:3,2469 Rayburn House Office Building,202-225-3261,None,AL,NaN,NaN,NaN,4.0,96.6,3.28,3,False,0103


In [10]:
members_pp['icpsr'] = members_pp['icpsr_id'].astype('float')

In [11]:
# you shouldn't merge more than 2 tables at once

members_total = pd.merge(members_vv,members_pp,
                        on = 'icpsr',
                        how = 'outer',
                        #validate = 'one_to_one',
                        indicator = 'matched')
members_total

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,bioguide_id,born,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2,id,title,short_title,...,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,senate_class,state_rank,lis_id,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,district,at_large,geoid,matched
0,117.0,President,99912.0,99.0,0.0,USA,200.0,0.0,0.0,"TRUMP, Donald John",NaN,1946.0,NaN,0.403,0.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,117.0,President,99913.0,99.0,0.0,USA,100.0,0.0,0.0,"BIDEN, Joseph Robinette, Jr.",B000444,1942.0,NaN,-0.320,0.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,117.0,House,20301.0,41.0,3.0,AL,200.0,NaN,NaN,"ROGERS, Mike Dennis",R000575,1958.0,NaN,0.360,0.463,-74.23598,0.89924,699.0,24.0,NaN,0.422,0.539,R000575,Representative,Rep.,...,True,NaN,0.360,None,22,2022,925.0,37.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:al/cd:3,2469 Rayburn House Office Building,202-225-3261,None,AL,NaN,NaN,NaN,4.00,96.60,3.28,3,False,0103,both
3,117.0,House,21102.0,41.0,7.0,AL,100.0,NaN,NaN,"SEWELL, Terri",S001185,1965.0,NaN,-0.393,0.402,-22.40082,0.96907,713.0,9.0,NaN,-0.498,0.489,S001185,Representative,Rep.,...,True,NaN,-0.393,None,12,2022,925.0,17.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:al/cd:7,2201 Rayburn House Office Building,202-225-2665,None,AL,NaN,NaN,NaN,1.84,98.34,1.55,7,False,0107,both
4,117.0,House,21193.0,41.0,5.0,AL,200.0,NaN,NaN,"BROOKS, Mo",B001274,1954.0,NaN,0.646,-0.404,-141.85497,0.81727,703.0,66.0,NaN,0.684,-0.270,B001274,Representative,Rep.,...,True,NaN,0.646,None,12,2022,925.0,26.0,0.0,2022-10-13 09:30:09 -0400,ocd-division/country:us/state:al/cd:5,2185 Rayburn House Office Building,202-225-4801,None,AL,NaN,NaN,NaN,2.81,84.79,15.10,5,False,0105,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W000826,Representative,Rep.,...,True,None,NaN,None,6,2022,925.0,9.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:pa/cd:7,1027 Longworth House Office Building,202-225-6411,None,PA,NaN,NaN,NaN,0.97,97.26,2.63,7,False,4207,right_only
722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W000788,Representative,Rep.,...,True,None,NaN,None,2,2022,925.0,1.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:ga/cd:5,1406 Longworth House Office Building,202-225-3801,None,GA,NaN,NaN,NaN,0.11,98.92,0.98,5,False,1305,right_only
723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W000827,Representative,Rep.,...,False,None,NaN,None,4,2022,31.0,15.0,0.0,2022-09-30 14:03:41 -0400,ocd-division/country:us/state:tx/cd:6,1725 Longworth House Office Building,202-225-2002,None,TX,NaN,NaN,NaN,48.39,93.33,0.00,6,False,4806,right_only
724,NaN,NaN,15432.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M000687,Representative,Rep.,...,True,None,NaN,None,4,2022,925.0,18.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:md/cd:7,2263 Rayburn House Office Building,202-225-4741,None,MD,NaN,NaN,NaN,1.95,98.78,1.10,7,False,2407,right_only


In [12]:
members_total['matched'].value_counts()

both          385
right_only    171
left_only     170
Name: matched, dtype: int64

In [13]:
pd.set_option('display.max_columns',80)
members_total.query("matched == 'right_only'")[members_pp.columns]

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,senate_class,state_rank,lis_id,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,district,at_large,geoid,icpsr
555,B001310,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Mike,None,Braun,None,1954-03-24,M,R,None,SenatorBraun,None,None,412839,None,148564,None,N00041731,/g/11f2ckdwty,S8IN00171,https://www.braun.senate.gov,None,http://www.braun.senate.gov/contact-mike,True,None,NaN,None,1,2024,866.0,24.0,0.0,2022-10-12 09:45:24 -0400,ocd-division/country:us/state:in,374 Russell Senate Office Building,202-224-4814,None,IN,1,junior,S397,2.77,90.97,9.03,NaN,NaN,NaN,NaN
556,H000601,"Senator, 2nd Class",Sen.,https://api.propublica.org/congress/v1/members...,Bill,None,Hagerty,None,1959-08-14,M,R,None,SenatorHagerty,None,None,456798,None,128466,None,N00045369,/g/11c1xp6h9v,S0TN00169,https://www.hagerty.senate.gov,https://www.hagerty.senate.gov/press-releases/...,https://www.hagerty.senate.gov,True,None,NaN,None,1,2026,866.0,16.0,0.0,2022-10-12 09:45:24 -0400,ocd-division/country:us/state:tn,B11 Russell Senate Office Building,202-224-4944,None,TN,2,junior,S407,1.85,91.90,8.10,NaN,NaN,NaN,NaN
557,H001089,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Joshua,None,Hawley,None,1979-12-31,M,R,None,SenHawleyPress,None,None,412840,None,169716,None,N00041620,/g/11bwy_95zy,S8MO00160,https://www.hawley.senate.gov,None,https://www.hawley.senate.gov/contact-senator-...,True,None,NaN,None,1,2024,866.0,15.0,0.0,2022-09-30 12:46:14 -0400,ocd-division/country:us/state:mo,212 Russell Senate Office Building,202-224-6154,None,MO,1,junior,S399,1.73,87.52,12.48,NaN,NaN,NaN,NaN
558,H000273,"Senator, 2nd Class",Sen.,https://api.propublica.org/congress/v1/members...,John,None,Hickenlooper,None,1952-02-07,M,D,None,SenatorHick,None,None,456797,None,None,None,N00044206,/m/04g_1z,S0CO00575,https://www.hickenlooper.senate.gov,None,https://www.hickenlooper.senate.gov,True,None,NaN,None,1,2026,866.0,1.0,0.0,2022-10-01 01:45:45 -0400,ocd-division/country:us/state:co,B85 Russell Senate Office Building,202-224-5941,None,CO,2,junior,S408,0.12,99.31,0.69,NaN,NaN,NaN,NaN
559,K000377,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Mark,None,Kelly,None,1964-02-21,M,D,None,SenMarkKelly,SenMarkKelly,SenMarkKelly,456794,None,None,None,N00044223,/m/02wsxy,S0AZ00350,https://www.kelly.senate.gov,None,https://www.kelly.senate.gov,True,None,NaN,None,1,2022,866.0,12.0,0.0,2022-10-01 01:45:45 -0400,ocd-division/country:us/state:az,B40b Dirksen Senate Office Building,202-224-2235,None,AZ,3,junior,S406,1.39,97.30,2.70,NaN,NaN,NaN,NaN
560,L000594,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Kelly,None,Loeffler,None,1970-11-27,F,R,,SenatorLoeffler,None,None,456790,None,188718,None,N00046125,/g/11j3kgpjlx,S0GA00526,https://www.loeffler.senate.gov,None,https://www.loeffler.senate.gov,False,None,NaN,None,1,2022,2.0,0.0,0.0,2022-09-16 12:02:48 -0400,ocd-division/country:us/state:ga,131 Russell Senate Office Building,202-224-3643,None,GA,3,senior,S405,0.00,100.00,0.00,NaN,NaN,NaN,NaN
561,O000174,"Senator, 2nd Class",Sen.,https://api.propublica.org/congress/v1/members...,Jon,None,Ossoff,None,1987-02-16,M,D,None,SenOssoff,None,None,456857,None,None,None,N00040675,/g/11g6njlbw2,S8GA00180,https://www.ossoff.senate.gov,https://www.ossoff.senate.gov/media/press-rele...,None,True,None,NaN,None,1,2026,864.0,39.0,0.0,2022-09-29 09:46:52 -0400,ocd-division/country:us/state:ga,None,None,None,GA,2,,S414,4.51,98.42,1.58,NaN,NaN,NaN,NaN
562,P000145,"Senator, 3rd Class",S

In [14]:
members_pp.columns

Index(['id', 'title', 'short_title', 'api_uri', 'first_name', 'middle_name',
       'last_name', 'suffix', 'date_of_birth', 'gender', 'party',
       'leadership_role', 'twitter_account', 'facebook_account',
       'youtube_account', 'govtrack_id', 'cspan_id', 'votesmart_id',
       'icpsr_id', 'crp_id', 'google_entity_id', 'fec_candidate_id', 'url',
       'rss_url', 'contact_form', 'in_office', 'cook_pvi', 'dw_nominate',
       'ideal_point', 'seniority', 'next_election', 'total_votes',
       'missed_votes', 'total_present', 'last_updated', 'ocd_id', 'office',
       'phone', 'fax', 'state', 'senate_class', 'state_rank', 'lis_id',
       'missed_votes_pct', 'votes_with_party_pct', 'votes_against_party_pct',
       'district', 'at_large', 'geoid', 'icpsr'],
      dtype='object')

In [15]:
members_vv.columns

Index(['congress', 'chamber', 'icpsr', 'state_icpsr', 'district_code',
       'state_abbrev', 'party_code', 'occupancy', 'last_means', 'bioname',
       'bioguide_id', 'born', 'died', 'nominate_dim1', 'nominate_dim2',
       'nominate_log_likelihood', 'nominate_geo_mean_probability',
       'nominate_number_of_votes', 'nominate_number_of_errors', 'conditional',
       'nokken_poole_dim1', 'nokken_poole_dim2'],
      dtype='object')

In [16]:
members_vv[['bioname','born','state_abbrev']].head(5)

,bioname,born,state_abbrev
0,"TRUMP, Donald John",1946,USA
1,"BIDEN, Joseph Robinette, Jr.",1942,USA
2,"ROGERS, Mike Dennis",1958,AL
3,"SEWELL, Terri",1965,AL
4,"BROOKS, Mo",1954,AL


In [17]:
members_pp[['last_name','date_of_birth','state']].head(5)

,last_name,date_of_birth,state
0,Baldwin,1962-02-11,WI
1,Barrasso,1952-07-21,WY
2,Bennet,1964-11-28,CO
3,Blackburn,1952-06-06,TN
4,Blumenthal,1946-02-13,CT


In [18]:
#members_pp.groupby(['last_name','date_of_birth']).value_counts().sort_value(ascending=False)

AttributeError: 'Series' object has no attribute 'sort_value'

# Steps:

1. Extract last name from bioname in members_vv
2. Convert last_name in members_pp to all caps
3. Extract the year from date_of_birth in members_pp
4. Check whether(last name, year) is unique in both members_vv and members_pp
5. If so, merge on (last name, year).  If not, try with (last name, year, state).

In [19]:
#members_vv[['bioname']].split(",")

AttributeError: 'DataFrame' object has no attribute 'split'

In [20]:
members_vv['last_name'] = members_vv['bioname'].str.split(pat=',',expand=True)[0]

In [21]:
members_pp['last_name'] = members_pp['last_name'].str.upper()

In [33]:
members_vv['last_name'] = members_vv['last_name'].str.upper()

In [34]:
members_pp['born'] = members_pp['date_of_birth'].str.split(pat="-",expand=True)[0]

In [35]:
members_vv[['last_name','born']].duplicated().value_counts()

False    552
True       3
dtype: int64

In [36]:
members_vv[['last_name','born','state_abbrev']].duplicated().value_counts()

False    555
dtype: int64

In [37]:
members_pp[['last_name','born','state']].duplicated().value_counts()

False    556
dtype: int64

In [38]:
members_vv['born'] = members_vv['born'].astype('str')
members_pp['born'] = members_pp['born'].astype('str')

In [39]:
members_total = pd.merge(members_vv,members_pp,
                        left_on = ['last_name','born','state_abbrev'],
                        right_on = ['last_name','born','state'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')

In [40]:
members_total['matched'].value_counts()

both          547
right_only      9
left_only       8
Name: matched, dtype: int64

In [41]:
members_total.query("matched == 'right_only'")[['last_name','born','state']]

,last_name,born,state
555,FERNANDEZ,1959,NM
556,FULCHER,1973,ID
557,GIMÉNEZ,1954,FL
558,GONZÁLEZ-COLÓN,1976,PR
559,OMAR,1981,MN
560,PELTOLA,1973,AK
561,PFLUGER,1978,TX
562,RYAN,1982,NY
563,SEMPOLINSKI,1983,NY


In [42]:
members_total.query("matched == 'left_only'")[['last_name','born','state_abbrev']]

,last_name,born,state_abbrev
0,TRUMP,1946,USA
1,BIDEN,1942,USA
116,GIMENEZ,1954,FL
137,FULCHER,1962,ID
224,OMAR,1982,MN
266,LEGER FERNANDEZ,1959,NM
356,GONZÁLEZ-CÓLON,1976,PR
404,PFLUGER,1977,TX
